<a href="https://colab.research.google.com/github/RobelDawit/code-store/blob/main/Scrape_coinmarketcap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Selenium

In [ ]:
!pip install --quiet selenium==4.10.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00


In [ ]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF


Executing: /tmp/apt-key-gpghome.5UKuwQIi9Y/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.OW7vRMzSiI/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.mZfuKal5SA/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [ ]:
!apt-get update -q
!apt-get install chromium chromium-driver -q

Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]
Get:4 http://deb.debian.org/debian-security buster/updates InRelease [34.8 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://deb.debian.org/debian buster/main amd64 Packages [10.7 MB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:10 http://deb.debian.org/debian buster-updates/main amd64 Packages [9,745 B]
Hit:11 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:14 http://deb.debian.org/de

# Scrape code

In [ ]:
from selenium import webdriver
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--incognito")
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920x1080")
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--enable-javascript')
    driver = webdriver.Chrome(options=options)
    return driver

from selenium.webdriver.common.by import By
from IPython.display import Image, display
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
coins_data = []
driver = web_driver()

In [ ]:
import time
driver.get("https://coinmarketcap.com/")

def scroll_down(driver):
    page_height = driver.execute_script("return document.body.scrollHeight")
    total_scrolled = 0
    for i in range(page_height):
        driver.execute_script(f'window.scrollBy(0,{i});')
        total_scrolled += i
        if total_scrolled >= page_height/2:
            last_no = i
            break
    for i in range(last_no, 0, -1):
        driver.execute_script(f'window.scrollBy(0,{i});')

def scroll_up(driver):
    page_height = driver.execute_script("return document.body.scrollHeight")
    total_scrolled = 0
    for i in range(0, -page_height, -1):
        driver.execute_script(f'window.scrollBy(0,{i});')
        total_scrolled += i
        if total_scrolled <= -page_height/2:
            last_no = i
            break
    for i in range(last_no, 0):
        driver.execute_script(f'window.scrollBy(0,{i});')

scroll_down(driver)
time.sleep(2)
scroll_up(driver)

html_source_code = driver.execute_script("return document.body.innerHTML;")
with open('html_source_code.html', 'w') as f:
    f.write(html_source_code)

# Find the table rows for the top coins
row_container = driver.find_elements(By.CLASS_NAME, "sc-14cb040a-3.dsflYb.cmc-table")
table = row_container[0].find_elements(By.TAG_NAME, "tbody")
rows = table[0].find_elements(By.TAG_NAME, "tr")

for row in rows[:100]:
  cells = row.find_elements(By.TAG_NAME, "td")
  if len(cells) >= 10:  # Check if the row has at least 10 cells
    coin_name = cells[2].find_element(By.TAG_NAME, "a").text
    coin_price = cells[3].text
    coin_market_cap = cells[7].text
    coin_volume = cells[8].text
    coin_circulating = cells[9].text
    coin = {
        "coin_name": coin_name,
        "coin_price": coin_price,
        "coin_market_cap": coin_market_cap,
        "coin_volume": coin_volume,
        "coin_circulating_supply": coin_circulating
    }
    coins_data.append(coin)
  else:
    print(f"Skipping row with {len(cells)} cells")
    for item in cells:
      print(item.text)
    print("------")


In [ ]:
driver.close()

import pandas as pd
df = pd.DataFrame(coins_data)

df = df.replace(r'\n', '', regex=True)
df

,coin_name,coin_price,coin_market_cap,coin_volume,coin_circulating_supply
0,BitcoinBTC,"$67,852.32","$1,335,047,099,635","$34,283,491,795506,149 BTC","19,674,268 BTC"
1,EthereumETH,"$3,326.13","$399,430,291,134","$15,378,186,0064,634,842 ETH","120,068,152 ETH"
2,Tether USDtUSDT,$1.00,"$106,625,907,464","$65,182,577,78965,170,063,136 USDT","106,601,457,935 USDT"
3,BNBBNB,$580.84,"$86,859,891,569","$2,021,609,4433,481,058 BNB","149,535,239 BNB"
4,SolanaSOL,$175.20,"$77,969,503,308","$7,251,296,69141,540,565 SOL","444,922,103 SOL"
...,...,...,...,...,...
95,KavaKAVA,$0.9022,"$977,148,095","$23,483,73726,051,452 KAVA","1,082,860,629 KAVA"
96,SATS1000SATS,$0.0004596,"$965,005,264","$32,975,75271,904,603,770 1000SATS","2,100,000,000,000 1000SATS"
97,IOTAIOTA,$0.2997,"$962,143,738","$17,321,89957,975,894 IOTA","3,210,367,601 IOTA"
98,PancakeSwapCAKE,$3.85,"$948,787,603","$131,879,33134,371,062 CAKE","246,576,313 CAKE"


In [ ]:
top100_coins = df.to_csv('top100_coins.csv', index = False)